# ctscan_mobilenetV2

Source: https://www.kaggle.com/code/ahmedtronic/covid-19-binary-classification-densenet169-98

## Import Libraries

In [ ]:
import os
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
import torch.nn as nn
from torch import optim
from torchvision import models, datasets

import warnings
warnings.filterwarnings("ignore")

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

device

## Helper Functions

In [ ]:
def learning_curves_tuning(train_loss, valid_loss, train_acc, valid_acc,
                           fine_tune_epoch=None):
    
    # Get training and validation data from initial training
    tacc = train_acc
    tloss = train_loss
    vacc = valid_acc
    vloss = valid_loss

    total_epochs = [i+1 for i in range(len(tacc))]

    # Find best epoch based on validation loss and accuracy
    index_loss = np.argmin(vloss)  # epoch with the lowest validation loss
    val_lowest = vloss[index_loss]
    
    index_acc = np.argmax(vacc)  # epoch with the highest validation accuracy
    acc_highest = vacc[index_acc]

    # Define plot labels
    sc_label = 'best epoch= ' + str(index_loss + 1)
    vc_label = 'best epoch= ' + str(index_acc + 1)

    plt.style.use('fivethirtyeight')
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

    # Plot loss curves
    axes[0].plot(total_epochs, tloss, 'r', label='Training Loss')
    axes[0].plot(total_epochs, vloss, 'g', label='Validation Loss')
    axes[0].scatter(index_loss + 1, val_lowest, s=150, c='blue', label=sc_label)

    # Add fine-tuning marker
    if fine_tune_epoch:
        axes[0].axvline(x=fine_tune_epoch, color='orange', linestyle='--',
                        label='Start Fine Tuning')

    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].set_ylim(0, 1)
    axes[0].legend()

    # Plot accuracy curves
    axes[1].plot(total_epochs, tacc, 'r', label='Training Accuracy')
    axes[1].plot(total_epochs, vacc, 'g', label='Validation Accuracy')
    axes[1].scatter(index_acc + 1, acc_highest, s=150, c='blue', label=vc_label)

    # Add fine-tuning marker
    if fine_tune_epoch:
        axes[1].axvline(x=fine_tune_epoch, color='orange', linestyle='--',
                        label='Start Fine Tuning')

    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def train_epoch(model):
    model.train()
    epoch_loss = 0
    total_correct = 0
    total_predictions = 0
    
    for batch_number, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        #forward path
        output = model(images)
        loss = criterion(output, labels)
        epoch_loss += loss.item()
        
        _, predicted = torch.max(output, 1)
        total_correct += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
                
        #bacward path
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_number and batch_number % 50 == 0):
            current_loss = epoch_loss / (batch_number + 1)
            current_acc = total_correct / total_predictions
            print(f"Batch [{batch_number}/{len(train_loader)}], Loss:{current_loss:0.6f}, Accuracy: {current_acc*100:.3f}%")
            
    avg_loss = epoch_loss / len(train_loader)
    avg_acc = total_correct / total_predictions
    
    return avg_loss, avg_acc

In [ ]:
def get_val(model):
    model.eval()
    epoch_loss = 0
    total_correct = 0
    total_predictions = 0
    
    with torch.no_grad():
    # no gradient cal for tensor, same for: requires_grad = False and detach()
        for batch_num, (images, labels) in enumerate(valid_loader):
    
            images = images.to(device)
            labels = labels.to(device)
            
            #forward path
            output = model(images)
            loss = criterion(output, labels)
            epoch_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total_correct += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
            
    avg_loss = epoch_loss / len(valid_loader)
    avg_acc = total_correct / total_predictions

    return avg_loss, avg_acc

In [ ]:
def get_test(model):
    """
    Calculates the test loss and accuracy for the given model.

    Parameters:
    - model: Trained model for evaluation.
    
    Returns:
    - test_loss: Average test loss over the test dataset.
    - test_accuracy: Test accuracy over the test dataset.
    """
    model.eval()
    epoch_loss = 0
    total_correct = 0
    total_predictions = 0

    with torch.no_grad():
        for batch_num, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            output = model(images)
            _, predicted = torch.max(output, 1)
            loss = criterion(output, labels)
            epoch_loss += loss.item()

            total_correct += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    # Average test loss
    avg_loss = epoch_loss / len(test_loader)
    avg_acc = total_correct / total_predictions
    
    return avg_loss, avg_acc

In [ ]:
def get_predict(model, data_loader, device):
    """
    Get all predictions and true labels from the data loader.

    Parameters:
    - model: Trained model for predictions.
    - data_loader: DataLoader for the dataset.

    Returns:
    - y_true: List of true labels.
    - y_pred: List of predicted labels.
    """
    model.eval()
    true_labels = []
    predictions = []

    with torch.no_grad():
        for batch_num, (images, labels) in enumerate(data_loader):
            
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            true_labels.extend(labels.cpu().numpy())
            predictions.extend(preds.cpu().numpy())

    return true_labels, predictions

## Load Data

In [ ]:
dataset_dir = "datasets/ctscan/raw"
covid_dir = "datasets/ctscan/raw/COVID"
non_covid_dir = "datasets/ctscan/raw/non-COVID"

image_size = (224, 224)
dataset = datasets.ImageFolder(root=dataset_dir)
class_names = dataset.classes
num_classes = len(class_names)
class_names

In [ ]:
dataset_paths = glob.glob(f"{dataset_dir}/**/*.png")
covid_paths = glob.glob(f"{covid_dir}/*.png")
non_covid_paths = glob.glob(f"{non_covid_dir}/*.png")

print("Total:", len(dataset_paths))
print("Covid:", len(covid_paths))
print("Non-Covid:", len(non_covid_paths))

In [ ]:
train_paths, test_paths = train_test_split(dataset_paths, test_size=0.15, random_state=42)

In [ ]:
val_paths, test_paths = train_test_split(test_paths, test_size=0.5, random_state=42)

In [ ]:
print("Train:", len(train_paths))
print("Val:", len(val_paths))
print("Test:", len(test_paths))

In [ ]:
covid_path = os.path.join("COVID", "Covid")

# Count the number of covid and non_covid in the validation set
train_covid_count = len([path for path in train_paths if covid_path in path])
train_non_covid_count = len([path for path in train_paths if "Non-Covid" in path])

# Count the number of covid and non_covid in the validation set
valid_covid_count = len([path for path in val_paths if covid_path in path])
valid_non_covid_count = len([path for path in val_paths if "Non-Covid" in path])

# Count the number of covid and non_covid in the test set
test_covid_count = len([path for path in test_paths if covid_path in path])
test_non_covid_count = len([path for path in test_paths if "Non-Covid" in path])

# Print the counts
print("Train set - Covid:", train_covid_count)
print("Train set - Non-Covid:", train_non_covid_count)
print("Validation set - Covid:", valid_covid_count)
print("Validation set - Non-Covid:", valid_non_covid_count)
print("Test set - Covid:", test_covid_count)
print("Test set - Non-Covid:", test_non_covid_count)

In [ ]:
torch.manual_seed(42)
random.seed(42)
random.shuffle(train_paths)

In [ ]:
img = Image.open(covid_paths[0])
img

In [ ]:
img = Image.open(non_covid_paths[0])
img

## Data Augmentation & Preprocessing

In [ ]:
train_transform = v2.Compose([
    v2.Resize(image_size),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomHorizontalFlip(p = 0.5), # 50 % from images will apply to
    v2.RandomVerticalFlip(p = 0.5), # 50 % from images will apply to
    v2.RandomRotation(10),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = v2.Compose([
    v2.Resize(image_size),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform = {
    "train_transform":train_transform,
    "valid_transform":valid_transform
}

In [ ]:
class CovidDataset(Dataset):
    def __init__(self, paths, transform = None, is_train =True):
        # data loadig
        self.paths = paths
        self.transform = transform
        self.is_train = is_train

    def __getitem__(self, index):
        img = Image.open(self.paths[index])
        img = img.convert("RGB") # Some images 4 channels
        label = self.paths[index][-15:-10]

        if self.transform:
            if self.is_train:
                img = self.transform["train_transform"](img)
            else:
                img = self.transform["valid_transform"](img)
        
        return img, (1 if label == "Covid" else 0)
        
    def __len__(self):
        return len(self.paths)

In [ ]:
train_data = CovidDataset(train_paths, transform)
valid_data = CovidDataset(val_paths, transform, is_train=False)
test_data = CovidDataset(test_paths, transform, is_train=False)

In [ ]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=16)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=16)
test_loader = DataLoader(test_data, shuffle=False, batch_size=16)

In [ ]:
next(iter(train_loader))[0].shape # images must have the same size

In [ ]:
next(iter(train_loader))[1]

In [ ]:
img = train_data[1][0]
plt.grid(False)
plt.axis(False)
plt.title("Covid" if train_data[1][1] == 1 else "Non-Covid")
plt.imshow(img.permute(1,2,0)); # img in tensor[channels, h, w]
# matplolib needs it [h, w, channels] so we use permute

## MobileNetV2

### Transfer Learning

In [ ]:
net = models.mobilenet_v2(weights='IMAGENET1K_V1')

In [ ]:
for parameter in net.parameters():
    parameter.requires_grad = False

net.classifier

In [ ]:
net.last_channel

In [ ]:
net.classifier = nn.Sequential(
    nn.Linear(net.last_channel, 512),  # Example: Add a fully connected layer
    nn.Dropout(p=0.5),
    nn.Linear(512, num_classes)  # Final output layer for binary classification (COVID vs. non-COVID)
)

net = net.to(device)

In [ ]:
# Early stopping parameters
best_score = float("inf") # ModelCheckpoint
patience = 0 # EarlyStopping

initial_epochs = 20  # Number of epochs before fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.classifier.parameters(), lr=1e-3, weight_decay=0.01)

In [ ]:
train_losses = []
valid_losses = []
train_accs = []
valid_accs = []

# train loop

for epoch in range(initial_epochs):
    print(f"Epoch[{epoch+1}/{initial_epochs}],    ", end="")

    avg_train_loss, train_acc = train_epoch(net)
    avg_valid_loss, valid_acc = get_val(net)

    # ModelCheckpoint & Save best model 
    if (avg_valid_loss < best_score):
        best_score = avg_valid_loss
        torch.save(net.state_dict(), f"model_net.pth")
        patience = 0
        
    # EarlyStopping
    else:
        patience += 1
        if (patience >= 10):
            print(f"Early stopping triggered after {epoch} epochs during training.")
            break
        
    train_losses.append(avg_train_loss)
    valid_losses.append(avg_valid_loss)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)

    print(f'Train Loss = {avg_train_loss:.6f} -- Train Accuracy = {train_acc*100:2.3f}%')
    print(f'Validation Loss = {avg_valid_loss:.6f} -- Validation Accuracy = {valid_acc*100:2.3f}%')
    print(" ")

## Fine-Tuning

In [ ]:
# net.features

In [ ]:
for parameter in net.features[-6:].parameters():
    parameter.requires_grad = True

In [ ]:
# Lower learning rate for fine-tuning
optimizer = optim.Adam(net.parameters(), lr=3e-5, weight_decay=0.01)

# Early stopping parameters
best_score = float("inf") # ModelCheckpoint
patience = 0 # EarlyStopping
fine_tuning_epochs = 50  # number of fine-tuning epochs

In [ ]:
# train loop

for epoch in range(fine_tuning_epochs):
    print(f"Epoch[{epoch+1}/{fine_tuning_epochs}],    ", end="")

    avg_train_loss, train_acc = train_epoch(net)
    avg_valid_loss, valid_acc = get_val(net)


    # ModelCheckpoint & Save best model 
    if (avg_valid_loss < best_score):
        best_score = avg_valid_loss
        torch.save(net.state_dict(), f"model_net_finetuned.pth")
        patience = 0
        
    # EarlyStopping Callback
    else:
        patience += 1
        if (patience >= 10):
            print(f"Early stopping triggered after {epoch} epochs during training.")
            break
    
        
    train_losses.append(avg_train_loss)
    valid_losses.append(avg_valid_loss)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)

    print(f'Train Loss = {avg_train_loss:.6f} -- Train Accuracy = {train_acc*100:2.3f}%')
    print(f'Validation Loss = {avg_valid_loss:.6f} -- Validation Accuracy = {valid_acc*100:2.3f}%')
    print(" ")

In [ ]:
learning_curves_tuning(train_losses, valid_losses, train_accs, valid_accs, fine_tune_epoch=initial_epochs)

In [ ]:
validation_loss, validation_accuracy = get_val(net)
print(f'Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy*100:.2f}%')

In [ ]:
test_loss, test_accuracy = get_test(net)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy*100:.2f}%')

## Classification Report & Confusion Matrix

In [ ]:
y_true, y_pred = get_predict(net, test_loader, device)
print(classification_report(y_true, y_pred, target_names= class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, class_names=class_names, figsize=(8,6));